# Import FabricEngineer Package

## Imports & Constants

In [ ]:
import json

In [ ]:
# Global IDs
GLOBAL_UTILS_WORKSPACE_ID = "b50e79c4-1140-479c-b5c1-225982ea6783"

# FabricEngineer
FABRICENGINEER_PACKAGE_ABFSS = "abfss://b50e79c4-1140-479c-b5c1-225982ea6783@onelake.dfs.fabric.microsoft.com/c7ab73ce-7e20-45c4-9bbf-004d37143759"
FABRICENGINEER_NOTEBOOK_NAME = "Export FabricEngineer-Py Modules"

## Functions

In [ ]:
def _get_module_code_from_lakehouse(module: str, version: str) -> str | None:
    filepath = f"{FABRICENGINEER_PACKAGE_ABFSS}/Files/py-packages/fabricengineer/{version}/{module}.py.txt"
    if not notebookutils.fs.exists(filepath):
        return None
    print("Code from lakehouse.")
    code = spark.read.text(filepath, wholetext=True).collect()[0][0]
    return code


def _get_module_code_from_notebook(module: str, version: str):
    print("Fetch code from notebook.")
    payload_str = notebookutils.notebook.run(
        path=FABRICENGINEER_NOTEBOOK_NAME,
        workspace=GLOBAL_UTILS_WORKSPACE_ID,
        arguments={
            "IS_DEV": False,
            "VERSION": version,
            "MODULES": module,
            "LAKEHOUSE_CODE_ABFSS": FABRICENGINEER_PACKAGE_ABFSS,
            "useRootDefaultLakehouse": True
        }
    )

    payload = json.loads(payload_str)
    return payload["modules"][module]


def get_module_code(module: str, version: str):
    code = _get_module_code_from_lakehouse(module, version)
    if code:
        return code
    return _get_module_code_from_notebook(module, version)


log = f"Passed functions {[get_module_code]}"
print(log)